# Two-Coin EM Example

We observe outcomes from 5 independent sequences of 10 coin tosses each. For each sequence, an **unobserved** choice is made between two biased coins, A and B, and then that chosen coin is tossed 10 times. We see the tosses but not which coin generated each sequence.

**Data (5 sequences):**

$X_1=`HTTTHHTHTH`=5\\
X_2=`HHHHTHHHHH`=9\\
X_3=`HTHTTTHHTT`=8\\
X_4=`HTHTTTHHTT`=4\\
X_5=`THHHTHHHTH`=7$

## Model

- Latent variable for sequence $i$:  
  $Z_i \in \{A, B\}$ indicating which coin was used.
- Parameters:
  - Coin usage frequency: $\pi = P(Z_i = A)$ so $1-\pi = P(Z_i = B) = 0.5$.
  - Coin frequencies of heads: $\theta_A = P(H \mid Z=A)$, $\theta_B = P(H \mid Z=B)$.

For sequence $i$, $X_i$ be the number of heads and $10-X_i$ the number of tails (e.g., count H/T in the string).

The (complete-data) likelihood for one sequence is:


$P(X_i| Z_i=A) = \theta_A^{X_i} (1-\theta_A)^{10-X_i},\\
P(X_i| Z_i=B) = \theta_B^{X_i} (1-\theta_B)^{10-X_i}.$

## EM Algorithm

**Initialize** $ \theta_A^{(0)}, \theta_B^{(0)}$ (e.g. $\theta_A^{(0)}=0.6, \theta_B^{(0)}=0.5$).

### E-step (compute expectations)

For each sequence $i$, compute the posterior probability that coin A (or B) generated it given theta from the $n$th step in the algorithm. In each step we need to find the posterior probablity of the latent state which is also denoted $q(Z_i)$:

$q(Z_i=A)=P(Z_i=A \mid X_i, \theta_A^{(n)}, \theta_B^{(n)})
= \frac{P(X_i|Z_i=A, \theta_A^{(n)}, \theta_B^{(n)})P(Z_i=A|\theta_A^{(n)}, \theta_B^{(n)})}
{\sum_zP(X_i|Z_i=z, \theta_A^{(n)}, \theta_B^{(n)})P(Z_i=z|\theta_A^{(n)}, \theta_B^{(n)})}
= \frac{(\theta_A^{(n)})^{X_i} (1-\theta_A^{(n)})^{10-X_i}\times 0.5}
{(\theta_A^{(n)})^{X_i} (1-\theta_A^{(n)})^{10-X_i}\times 0.5+(\theta_B^{(n)})^{X_i} (1-\theta_B^{(n)})^{10-X_i}\times 0.5}.$


Then $1-P(Z_i=A \mid X_i, \theta_A^{(n)}, \theta_B^{(n)})= P(Z_i=B  \mid X_i, \theta_A^{(n)})$.

### M-step (maximize expected complete-data log-likelihood)

Update parameters using weighted counts (weights = responsibilities):


$\theta_A^{(n+1)} =
\frac{\sum_{i=1}^N q(Z_i=A)^{(n)} X_i}
{\sum_{i=1}^N q(Z_i=A)^{(n)} (10)}, \quad
\theta_B^{(n+1)} =
\frac{\sum_{i=1}^N (1-q(Z_i=A)^{(n)}) X_i}
{\sum_{i=1}^N (1-q(Z_i=A)^{(n)}) (10)}.$

**Iterate** E-step and M-step until convergence of the log-likelihood or parameters.
here "(n)" refers to the $n$th step in the EM algorithm

## Applying to the Given Data

1. **Count heads/tails** for each sequence to get $(X_i, T_i)$ heads and 10 heads or tails.  
2. **Run EM** using the formulas above:  
   - E-step: compute $q(Z_i=A)^{(n)}$ for all 5 sequences.  
   - M-step: update $\theta_A, \theta_B$.  
3. **Repeat** until parameter changes are negligible (e.g., $<10^{-4}$).

## Notes

- EM is guaranteed to **not decrease** the data log-likelihood each iteration.  
- Different initializations can converge to slightly different local optima—try a few starts to check stability.



 - What are Z , X and θ  ?
 - What is P(X|Z,θ)
 - What is P(Z|X,θ)  ?
 - What is P(Z|θ)  ?


In [ ]:
EMcoinStep<-function(theta,data){
  lik <- cbind(#likelihood of data assuming
    dbinom(data,10,theta[1]), #coin A
    dbinom(data,10,theta[2]) #coin B
  )
  ##probabilty of coin given data
  wA<-lik[,1]*0.5 / (lik[,1]*0.5 + lik[,2]*0.5)
  wB<-lik[,2]*0.5 / (lik[,1]*0.5 + lik[,2]*0.5)
  newTheta <-c(#n+1 - parameter
    sum(data*wA)/sum(data*wA + (10-data)*wA),
    sum(data*wB)/sum(data*wB + (10-data)*wB)
  )
  return(newTheta)
}

data <- c(5,9,8,4,7)

theta <- c(0.5,0.6)
for(i in 1:100)
  print(theta<-EMcoinStep(theta,data))



try to run the algorithm in R
 - do you get the same results as in the review?
 - what is lik
 - where is P(z|θ) in the R code
 - what is wA and wB
 - what is sum(data*wA)
 - what is sum(data*wA + (10-data)*wA)


In the experiment their is an assumption that each coin will be used with equal probability. Lets try to remove this assumption and instead estimate the probability of using coin A and coin B

 - try to calculate the expected number of coin A and B tosses from the figure in the article.
try to modify the above code to also estimate the probability of using coin A and B.
 - how many additional parameters needs to be estimated?
 - implement the new model


### Bonus 1 - write the likelihood

 - try to write the formula for the likelihood you are maximizing


$p(X|\theta)$

### Bonus 2 - What if we don't know the coin probability
instead of assuming the coin probability is 0.5 we can estimate it from the data

In [ ]:
EMcoinStep2<-function(theta,data){
  pA <- theta[3]
  pB <- 1-pA
  lik <- cbind(#likelihood of data assuming
    dbinom(data,10,theta[1]), #coin A
    dbinom(data,10,theta[2]) #coin B
  )
  ##probabilty of coin given data
  wA<-lik[,1]*pA / (lik[,1]*pA + lik[,2]*pB)
  wB<-lik[,2]*pB / (lik[,1]*pA + lik[,2]*pB)
  newTheta <-c(#n+1 - parameter
    sum(data*wA)/sum(data*wA + (10-data)*wA),
    sum(data*wB)/sum(data*wB + (10-data)*wB),
    sum(wA)/length(data)
  )
  return(newTheta)
}

data <- c(5,9,8,4,7)
## theta is the freq_A, freq_B, p(coin=A)
theta <- c(0.6,0.5,0.9)
for(i in 1:100)
  print(theta<-EMcoinStep2(theta,data))

# EM algorithm examples

## Estimating allele frequencies from genotype likelihoods
We will try to estimate allele frequencies based on the reads in slides "Simple allele frequency estimator" from 6 individuals. The number of reads supporting allele G and allele T is shown in the table

| indivduals      | 1  | 2  | 3 | 4 | 5 | 6 |
|-------|----|----|---|---|---|---|
| G reads     | 7  | 25 | 5 | 4 | 0 | 0 |
| T reads    | 0  | 1  | 3 | 4 | 2 | 4 |

To simplify we will assume that the seqeuncing error rate is $\epsilon=0.01$ for all reads (Not recommended for real data) and we will use the GATK model to that $\epsilon_{G->T}=\epsilon_{T->G}=0.01/3$. When the error rates are the same for each reads and we only observe to kinds of read then the GATK model simplifies to

\begin{aligned}
g_{GG} &= (1 - \epsilon)^{\#G} \left(\tfrac{\epsilon}{3}\right)^{\#T}, \\
g_{GT} &= \left(\frac{1}{2}(1 - \epsilon) + \frac{1}{2} \tfrac{\epsilon}{3}\right)^{\#G+\#T}, \\
g_{TT} &= (1 - \epsilon)^{\#T} \left(\tfrac{e}{3}\right)^{\#G}.
\end{aligned}


You can read and run the calculatins in R below


In [ ]:
e <-0.01
Greads <- c(7,25,5,4,0,0)
Treads <- c(0,1,3,4,2,4)


# Genoetype likelyhoods. Ignore how the genotype likelihoods are calculated here
# genoetype likelihood assuming the error is the same for all reads
# NB!. For real data we have four allele and we cannot calculate it like this because the error rate is different for each read
gGG <- (1-e)^Greads * (e/3)^Treads
gGT <-  ( 0.5*(1-e) + 0.5 * e/3 )^(Treads+Greads)
gTT <- (1-e)^Treads * (e/3)^Greads
GL<-rbind(gGG,gGT,gTT)
colnames(GL) <- paste("ind",1:6)

cat("Genotypes likehooods:\n")
print(GL)

To estimate the allele frequency we can write up the likelihood as
$$p(X \mid \theta) = \prod_{i=1}^N p(X_i|\theta) = \prod_{i=1}^N\sum_{z}p(X_i|Z_i=z)p(Z_i=z|\theta)$$
 - What is N and i?
 - What are Z, X and $\theta$  ?
 - What is $P(X_i|Z_i=z)$?
 - What is $P(Z_i=z|\theta)$  ?
 - What are we assuming by setting $p(X \mid \theta) = \prod_{i=1}^N p(X_i|\theta)$?



Let try to calculate the likelhoods assuming that the allele frequency is 0.3 or 0.1

In [ ]:
calculateLogLike <- function(theta,GL){
   #p(z|θ)
   pZ  <- c(pZ0= (1-theta)^2 ,pZ1= 2*theta*(1-theta), pZ2= theta^2 )
   sum( log( colSums(GL*pZ) ))
}

theta <- 0.3
cat("\n log-likelhoods with theta_T",theta,"\n")
calculateLogLike(theta , GL)

theta <- 0.1
cat("\n log-likelhoods with theta_T",theta,"\n")
calculateLogLike(theta , GL)

 - Which frequency fits best with the data?
 - Identify the genotype likelihoods $P(X_i|Z_i=z)$ and the use of HWE $P(Z_i=z|\theta)$ in the code.

Lets plot the likelihood surface

In [ ]:
thetas <- 1:99/100
ll <- sapply(
  thetas,
  function(x) calculateLogLike(x,GL)
)
plot(thetas,ll,ylab="Log likelihood",xlab="Frequency (theta_T)",type="l")

To find the maximum we use a EM algorithm which has two steps

E-step:
   $q_i(Z_j)=p(Z_j|X_i,\theta^{n})$
   

M-step:
     $\theta^{(n+1)} = argmax_{\theta} Q(\theta|\theta^{(n)})$
   
Therefore in each in each iteration of the EM algorith we need to calculate the posterior probabiltiy of the genotype given the frequency. For this we use bayes formula
$$p(Z_i=z|X_i,\theta^{(n)}) =\frac{p(X_i|Z_i=z,\theta^{(n)})p(Z_i=z|\theta^{(n)}) }{\sum_{z'}p(X_i|Z_i=z’,\theta^{(n)})p(Z_i=z’|\theta^{(n)})}$$
which is can simply by
$p(X_i|Z_i=z,\theta^{(n)})=p(X_i|Z_i=z)$
 - Why can we do that?

In the second part we use the posterior probabilities to get the expected number of G and T alleles.

The expected fraction of G alleles is
$$\frac{\sum_i^N 2\times P(Z_i=GG|X_i,\theta^{(n)})+P(Z_i=GT|X_i,\theta^{(n)})}{\sum_z \sum_i^N 2\times P(Z_i=z|X_i,\theta^{(n)})}=\frac{\sum_i^N 2\times P(Z_i=GG|X_i,\theta^{(n)})+P(Z_i=GT|X_i,\theta^{(n)})}{12}$$
 - Explain the 2 and the 12 in the above equations.




 Lets run the EM algorithm

In [ ]:

EMgeno<-function(theta,GL){ #numeric optimazition EM
  ## Q(Z), for Z = GG,GT,TT = 0,1,2
  #p(x|z)p(z|θ)
   w0<-GL[1,]*(1-theta)^2
   w1<-GL[2,]*2*theta*(1-theta)
   w2<-GL[3,]*theta^2

  ## p(z|θ,x)
  # p(x|θ)= \sum_z p(x|z)p(z|θ) =
  # p(x|Z=0)p(Z=0|θ)+p(x|Z=1)p(Z=1|θ)+p(x|Z=2)p(Z=2|θ)
   pX <- w0 + w1 + w2
  # p(z|θ,x) = p(x|z) p(z|θ) / p(x|θ)
   p0 <- w0/pX
   p1 <- w1/pX
   p2 <- w2/pX

  ## Î¸^(n+1) = E[allele 1] / ( E[allele 1] + E[allele 2] )
   thetaNew <- sum( p1+2*p2 ) / sum( 2*p0 + 2*p1 + 2*p2 )
   return(thetaNew)
}


maxIter <- 10
output <- matrix(NA,nrow=maxIter+1,ncol=3)
colnames(output)<- c("iter","theta","LL")
#starting point
theta <- c(0.2)

for(i in 0:maxIter){
  #calculate likelihood and print (not used by the EM)
  output[i+1,] <- c(i,theta,calculateLogLike(theta,GL))
  cat("theta=",output[i+1,"theta"],"logLike=",output[i+1,"LL"],"iteration",output[i+1,"iter"],"\n")

  #update theta (the EM steps)
  theta <- EMgeno(theta,GL)
}
output

In [ ]:
plot(thetas,ll,ylab="Log likelihood",xlab="Frequency (theta)",type="l")
points(output[,"theta"],output[,"LL"],pch=16,col=1:nrow(output))
text(output[,"theta"],output[,"LL"]*1.03,output[,"iter"],col=1:nrow(output))

plot(thetas,ll,ylab="Log likelihood",xlab="Frequency (theta)",type="l",
ylim=range(output[-1,"LL"]),xlim=range(output[-1,"theta"]),main="zoom")

points(output[,"theta"],output[,"LL"],pch=16,col=1:nrow(output))
text(output[,"theta"],output[,"LL"]*1.00008,output[,"iter"],col=1:nrow(output))


#abline(v=theta_save)


 - how many iterations are needed for convergense?
 - try a different starting point the is further away from the maximum likelihood


### Bonus ( if you have extra time)
 - test if the site is polymorphic (hint: calc the likelihoods and use 1-pchisq(x,1) to get a p-value)


# Coin toss - how to find the maximum likelihood for a binomial

Let $X\in \{0,1\}^N$ be the outcome of a series of $N$ coin tosses  (the data) where each 0 corresponds to the outcome tail and each 1 is the outcome of a head. To estimate the frequency ($\theta$) we can write up the likelihood as
$$
\begin{aligned}
P(X|\theta)&=Binom(k=\sum X;p=\theta,n=N)={N\choose k} \prod_{i=1}^N P(X_i|\theta)\\
&={N\choose k} \prod_{i=1}^N \theta^{X_i}(1-\theta)^{1-X_i}∝\prod_{i=1}^N \theta^{X_i}(1-\theta)^{1-X_i}=\prod_{i=1}^N P(X_i|\theta)
\end{aligned}
$$

To find the maximum likelihood we can choose to maximize with or without the binomial coefficient

In [ ]:
## simulate 10 coin tosses
theta_true <- 0.5
( X <- rbinom(10,1,theta_true) )

k <- sum(X)

cat("Number of heads",k,"\n")

To find the maximum we can work on any function that is proportional to the probability of the data which is called the likelihood. We can also choose to work on the log-likelihood. As can be seen below they will have the maximum the same place.

In [ ]:
thetas <- 1:99/100

likBinom <- dbinom(k,10,thetas)
plot(thetas,likBinom,ylab="Binom likelihood",xlab="Frequency (theta)",type="l",main="Likelihood - Binom(k=sum X,p=theta,n=N) ")
abline(v=k/10,col="red");legend("topright","Maximum\n likelihood",lty=1,col="red")

likProd <- sapply(thetas, function(x) prod(c(1-x,x)[X+1]) )
plot(thetas,likProd,ylab="prod likelihood",xlab="Frequency (theta)",type="l",main="Likelihood - without binomial coefficient")
abline(v=k/10,col="red");legend("topright","Maximum\n likelihood",lty=1,col="red")

plot(thetas,log(likBinom),ylab="Binom log-likelihood",xlab="Frequency (theta)",type="l",main="log-Likelihood - Binom(k=sum X,p=theta,n=N) ")
abline(v=k/10,col="red");legend("topright","Maximum\n likelihood",lty=1,col="red")


### Analytical solution
$$P(X|\theta)={N\choose k} \prod_{i=1}^N P(X_i|\theta)={N\choose k}\theta^k(1-\theta)^{N-k},$$
where $k=\sum_i X_i$.
The log likelhood is
$$LL(\theta)=log(P(X|\theta))=log{N\choose k} +\sum_{i=1}^N log  \left(P(X_i|\theta)\right)=log{N\choose k}+k\cdot log(\theta) +(N-k)log(1-\theta).$$

To find the maximum we set the derivative to zero

$$\frac{d LL(\theta)}{d \theta}=\frac{k}{\theta}-\frac{N-k}{1-\theta}=0 \quad \Rightarrow \quad \theta=\frac{k}{N}$$
therefore the maximum likelihood solution is
$$\hat{\theta}=argmax_{\theta}{N\choose k}\theta^k(1-\theta)^{N-k}=\frac{k}{N}$$



